In [1]:
import pandas as pd
import csv
import math

# Reading the input file

In [2]:
inputfile = 'unknownDiaInputRandom.csv'

df = pd.read_csv(inputfile)
df

,Power,Velocity Ratio,N1,N2,z1,z2,S_d1,S_d2,material
0,10,1.5,200,NaN,10,NaN,172.6,172.6,steel-steel
1,10,1.5,200,NaN,10,NaN,172.6,220.6,steel-steel
2,10,1.5,200,NaN,10,NaN,172.6,207.0,steel-steel
3,10,1.5,200,NaN,10,NaN,172.6,233.4,steel-steel
4,10,1.5,200,NaN,10,NaN,172.6,462.0,steel-steel
...,...,...,...,...,...,...,...,...,...
701995,55,3.5,1400,NaN,39,NaN,516.8,220.6,steel-steel
701996,55,3.5,1400,NaN,39,NaN,516.8,207.0,steel-steel
701997,55,3.5,1400,NaN,39,NaN,516.8,233.4,steel-steel
701998,55,3.5,1400,NaN,39,NaN,516.8,462.0,steel-steel


# Helper Functions

In [3]:
def form_factor(z):
    return (0.154 - (0.912 / int(z)))

def m_standard(m):
    STANDARD_VALUES = [1.0, 1.25, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 6.0, 
                       8.0, 10.0, 12.0, 16.0, 20.0, 25.0, 32.0, 40.0, 50.0, 60.0, 80.0, 100.0]
    return next(value for value in STANDARD_VALUES if m < value)

def module(power, S_d, Y, z, N, Cv=0.5, k=10):
    Mt = (9.55 * 10**6 * power) / N
    return ((2 * Mt) / (S_d * Cv * k * Y * z)) ** (1/3)

def get_Cv(v):
    if v < 8: Cv = 3.05 / (3.05 + v)
    elif 8 <= v < 13: Cv = 4.58 / (4.58 + v)
    elif 13 <= v < 20: Cv = 6.1 / (6.1 + v)
    else: Cv = 5.55 / (5.55 + math.sqrt(v))
    return Cv

def get_error(v):
    velocity_error_map = {1.25: 0.0925,
                          2.50: 0.0800,
                          3.75: 0.0700,
                          5.00: 0.0600,
                          6.25: 0.0525,
                          7.50: 0.0475,
                          8.75: 0.0425,
                          10.0: 0.0375,
                          11.25: 0.0325,
                          12.50: 0.0300,
                          13.75: 0.0250,
                          15.00: 0.0225,
                          16.25: 0.0200,
                          17.25: 0.0170,
                          20.00: 0.0150,
                          22.50: 0.0150
                         }
    
    table = velocity_error_map
    
    if v in table:
        return table[v]
    
    if v < 1.25:
        v = 1.25
        return table[v]
    
    if v > 22.50:
        v = 22.50
        return table[v]

    v1 = math.floor(v)
    while v1 not in table:
        v1 -= 0.25
        if v1 < 1.25:
            v1 = 1.25
            break

    return table[v1]

def interpretation(error, table):
    if error in table:
        return table[error]

    if error < 0 or error > 1:
        raise ValueError('Error value not in range 0.01 < error < 0.05')

    E1 = math.floor(error*100)
    E2 = math.ceil(error*100)
    E1, E2 = round(E1/100, 4), round(E2/100, 4)
    while E1 not in table:
        E1 -= 0.005
        E1 = round(E1, 4)

    while E2 not in table:
        E2 += 0.005
        E2 = round(E2, 4)
        
    C1 = table[E1]
    C2 = table[E2]

    return ((error-E1) * (C2-C1) / (E2-E1)) + C1

def dynamic_factor(error, material_choice):
    error_range = [0.0125, 0.025, 0.050, 0.075, 0.100, 0.125]
    
    st_ci = [96.04, 192.08, 384.16, 576.34, 768.32, 960.40]
    ci_ci = [69.85, 139.70, 279.40, 419.10, 558.80, 698.50]
    st_st = [139.70, 279.48, 558.80, 838.20, 1117.60, 1397.00]
    
    material_map = {'steel-steel': st_st, 'steel-ci': st_ci, 'ci-ci': ci_ci}
    
    material = material_map[material_choice]
    mapping = dict(zip(error_range, material))
    return interpretation(error, mapping)

def get_BHN(s_d, material):    
    ci = {47.1: 200, 56.4: 225, 78.5: 300}
    st = {138.3: 180, 193.2: 250, 172.6: 150,
          220: 200, 220.6: 300, 207: 150, 
          233.4: 200, 345.2: 650, 462: 400, 
          516.8: 450
         }
    
    mapping = {47.1: 200, 56.4: 225, 78.5: 300, 
               138.3: 180, 193.2: 250, 68.7: 80, 
               82.4: 100, 152: 180, 172.6: 150,
               220: 200, 220.6: 300, 207: 150, 
               233.4: 200, 345.2: 650, 462: 400, 
               516.8: 450
              }
    
    materials = {'ci': ci, 'steel': st}
    keys = sorted(materials[material])
    s_d = next(key for key in keys if s_d <= key)

    return mapping[s_d]

def get_k_from_bhn(bhn1, bhn2, material):
    st_st = {(150, 150): 0.2070, (200, 150): 0.2962, (250, 150): 0.4002,
             (200, 200): 0.4002, (250, 200): 0.5239, (300, 200): 0.6622,
             (250, 250): 0.6622, (300, 250): 0.8211, (350, 250): 0.9928,
             (300, 300): 0.9928, (350, 300): 1.1792, (400, 300): 1.2822,
             (350, 350): 1.3862, (400, 350): 1.6069, (400, 400): 1.8482}

    st_ci = {(150, 180): 0.3031, (200, 180): 0.6004}

    ci_ci = {(180, 180): 1.0487}

    materials = {'steel-steel': st_st, 'steel-ci': st_ci, 'ci-ci': ci_ci}    
    
    bhn1s = sorted([key[0] for key in materials[material]])
    bhn2s = sorted([key[1] for key in materials[material]])
    
    try:
        bhn1 = next(val for val in bhn1s if bhn1 <= val)
    except Exception:
        bhn1 = bhn1s[-1]
    
    possible_values = [key[1] for key in materials[material] if key[0] == bhn1]
    
    try:
        bhn2 = next(val for val in possible_values if bhn2 <= val)
    except Exception:
        bhn2 = possible_values[-1]
    
    return materials[material][(bhn1, bhn2)]

def get_bhn_from_k(k, material):
    st_st = {0.2070: (150, 150), 0.2962: (200, 150), 0.4002: (250, 150),
             0.4002: (200, 200), 0.5239: (250, 200), 0.6622: (300, 200),
             0.6622: (250, 250), 0.8211: (300, 250), 0.9928: (350, 250),
             0.9928: (300, 300), 1.1792: (350, 300), 1.2822: (400, 300),
             1.3862: (350, 350), 1.6069: (400, 350), 1.8482: (400, 400)}

    st_ci = {0.3031: (150, 180), 0.6004: (200, 180)}

    ci_ci = {1.0487: (180, 180)}

    materials = {'steel-steel': st_st, 'steel-ci': st_ci, 'ci-ci': ci_ci}
    
    sorted_ks = sorted([k for k in materials[material]])
    try:
        k = next(val for val in sorted_ks if k <= val)
    except Exception:
        k = sorted_ks[-1]
    
    return materials[material][k]

# Function to compute the outputs for the given inputs

In [4]:
# solve for z1, z2, b
def get_outputs(row):
    power = row['Power']
    vr = row['Velocity Ratio']
    N1 = row['N1']
    N2 = row['N2']
    z1 = row['z1']
    z2 = row['z2']
    S_d1 = row['S_d1']
    S_d2 = row['S_d2']
    material = str(row['material']).lower()
    
    if math.isnan(N1):
        N1 = vr*N2
    
    if math.isnan(N2):
        N2 = N1/vr
    
    if math.isnan(vr):
        vr = N1/N2
    
    if math.isnan(z1):
        z1 = z2/vr
    
    if math.isnan(z2):
        z2 = vr*z1
    
    if math.isnan(S_d1):
        S_d1 = 233.4
    
    if math.isnan(S_d2):
        S_d2 = 233.4
    
    y1, y2 = form_factor(z1), form_factor(z2)
    
    if (S_d1 == S_d2) or ((S_d1 * y1) < (S_d2 * y2)):
        Y, z, S_d, N = y1 * math.pi, z1, S_d1, N1
    else:
        Y, z, S_d, N = y2 * math.pi, z2, S_d2, N2
    
    m = module(power, S_d, Y, z, N)
    m = m_standard(m)
    
    d1 = m*z1
    d2 = m*z2
    
    if (S_d1 == S_d2) or ((S_d1*y1) < (S_d2*y2)):
        d = d1
    else:
        d = d2
    
    b = 10*m
    
    Mt = (9.55 * 10**6 * power) / N
    Ft = 2*Mt / d
    v = (math.pi * d * N) / (60 * 1000)
    
    Cv = get_Cv(v)
    
    S_d_ind = Ft / (Cv * b * Y * m)
    
    if S_d_ind > S_d:
        m = m_standard(m)
        d1 = m*z1
        d2 = m*z2
        b = 10*m
    
    k3 = 20.67
    
    
    error = get_error(v)
    C = dynamic_factor(error, material)
    
    Fd = Ft + (k3 * v * (C*b + Ft)) / (k3*v + math.sqrt(C*b + Ft))
    Q = 2*d2 / (d2+d1)
    
    material1, material2 = material.split('-')
    roh = 1000
    weight = ((math.pi * d1**2 * b * roh) / (4 * 10**9)) + ((math.pi * d2**2 * b * roh) / (4 * 10**9))
    bhn1 = get_BHN(S_d1, material1)
    bhn2 = get_BHN(S_d2, material2)
    K = get_k_from_bhn(bhn1, bhn2, material)
    Fw = d1 * b * Q * K
    
    if Fw < Fd:
        K = Fd / (d1*b*Q)
        bhn1, bhn2 = get_bhn_from_k(K, material)
    
    
    
    # rounding to 4 decimal places
    z1 = int(z1)
    z2 = int(z2)
    weight = round(weight, 4)
    values = {'Power': power, 'Velocity Ratio': vr, 'N1': N1, 'N2': N2, 'z1': z1, 'z2': z2, 
              'S_d1': S_d1, 'S_d2': S_d2, 'material': material, 'd1': d1, 'd2': d2, 
              'b': b, 'BHN1': bhn1, 'BHN2': bhn2, 'weight': weight}
    return values

# Computing output and writing to csv file

In [ ]:
output_file = 'unknownDiaOutput.csv'

input_fields = ['Power', 'Velocity Ratio', 'N1', 'N2', 'z1', 'z2', 'S_d1', 'S_d2', 'material']
output_fields = ['d1', 'd2', 'b', 'BHN1', 'BHN2', 'weight']

with open(output_file, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=input_fields+output_fields)
    writer.writeheader()
    
    for index, row in df.iterrows():
        try:
            outputs = get_outputs(row)
            writer.writerow(outputs)
        except Exception as e:
            pass